In [1]:
"""
CHUNK 1: Estrutura Geral do Programa e Imports
Configuração inicial e carregamento do arquivo CSV principal
"""

import os
import pandas as pd
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv
import time
import json
from datetime import datetime
import logging
import re

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('agrupamento_projetos.log'),
        logging.StreamHandler()
    ]
)

# Carregar variáveis de ambiente
load_dotenv()

# Carregar arquivo CSV principal
arquivo_dados = 'csv_longo/projetos_lei_do_bem_JUSTIFICATIVAS_RESULTADOS_PESSOAS.csv'
df = pd.read_csv(arquivo_dados, sep=';', encoding='utf-8')

print(f"📊 Arquivo carregado: {len(df)} registros encontrados")
print(f"📋 Colunas disponíveis: {len(df.columns)}")
print(f"📄 Arquivo: {arquivo_dados}")

# Definir colunas baseadas na estrutura do CSV carregado
colunas_identificacao = [
    'id_empresa_ano', 'empresa', 'ano_referencia'
]

colunas_analise = [
    'setor', 'natureza', 'tipo_pesquisa', 'projeto', 'projeto_resultados'
]

colunas_avaliacao = [
    'do_resultado_analise', 'p_resultado_analise'
]

# Configurações globais
LIMITE_PROJETOS_POR_LOTE = 50
TEMPO_PAUSA_ENTRE_REQUESTS = 2
MAX_TENTATIVAS = 3

# Verificar se as colunas necessárias existem
colunas_faltando = [col for col in colunas_analise if col not in df.columns]
if colunas_faltando:
    print(f"⚠️ Colunas não encontradas: {colunas_faltando}")
    print(f"📋 Colunas disponíveis no CSV: {list(df.columns)}")
else:
    print("✅ Todas as colunas necessárias encontradas")

print("✅ Chunk 1 executado: Imports, configurações e dados carregados")
print(f"📁 Colunas de identificação: {colunas_identificacao}")
print(f"🔍 Colunas para análise: {colunas_analise}")
print(f"📊 Colunas de avaliação: {colunas_avaliacao}")

📊 Arquivo carregado: 75348 registros encontrados
📋 Colunas disponíveis: 50
📄 Arquivo: csv_longo/projetos_lei_do_bem_JUSTIFICATIVAS_RESULTADOS_PESSOAS.csv
✅ Todas as colunas necessárias encontradas
✅ Chunk 1 executado: Imports, configurações e dados carregados
📁 Colunas de identificação: ['id_empresa_ano', 'empresa', 'ano_referencia']
🔍 Colunas para análise: ['setor', 'natureza', 'tipo_pesquisa', 'projeto', 'projeto_resultados']
📊 Colunas de avaliação: ['do_resultado_analise', 'p_resultado_analise']


/tmp/ipykernel_1786516/2852269188.py:33: DtypeWarning: Columns (25,26,32,34,35,36,39,41,43,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_dados, sep=';', encoding='utf-8')


In [2]:
"""
CHUNK 1.5 (Temporário): Análise de Setores para Teste
Identifica os setores com o menor número de projetos para otimizar os testes.
"""

print("\n🔄 Executando análise de setores para encontrar o menor para testes...")

try:
    if 'df' in locals() and not df.empty:
        # Contar o número de projetos para cada valor único na coluna 'setor'
        contagem_setores = df['setor'].value_counts()
        
        # Ordenar os resultados do menor para o maior
        setores_menores = contagem_setores.sort_values(ascending=True)
        
        print("\n" + "="*60)
        print("📊 TOP 10 SETORES COM MENOS PROJETOS")
        print("="*60)
        # Exibe os 10 menores setores e suas contagens
        print(setores_menores.head(10).to_string())
        
        # Pega o nome e a contagem do menor setor
        if not setores_menores.empty:
            menor_setor_nome = setores_menores.index[0]
            menor_setor_contagem = setores_menores.iloc[0]
            
            print("\n" + "="*60)
            print("🏆 SUGESTÃO PARA TESTE")
            print("="*60)
            print(f"O setor com o menor número de projetos é: '{menor_setor_nome}'")
            print(f"Total de Projetos: {menor_setor_contagem}")
            print("\n💡 Dica: Copie o nome do setor acima e cole na variável 'CATEGORIA_TESTE_API' no Chunk 2.")
        else:
            print("⚠️ Não foi possível encontrar o menor setor.")

    else:
        print("❌ DataFrame 'df' não encontrado ou está vazio. Execute o Chunk 1 primeiro.")

except Exception as e:
    print(f"❌ Ocorreu um erro durante a análise dos setores: {e}")


🔄 Executando análise de setores para encontrar o menor para testes...

📊 TOP 10 SETORES COM MENOS PROJETOS
setor
Metalurgia e Mineração        5358
Eletroeletrônica              7474
Transversal                   8118
Agroindústria e Alimentos     8659
Mecânica e Transporte        10189
Química e Farmácia           14230
TIC                          20439

🏆 SUGESTÃO PARA TESTE
O setor com o menor número de projetos é: 'Metalurgia e Mineração'
Total de Projetos: 5358

💡 Dica: Copie o nome do setor acima e cole na variável 'CATEGORIA_TESTE_API' no Chunk 2.


In [3]:
"""
CHUNK 2: Preparação dos Dados
Carrega o CSV e prepara os dados para processamento, incluindo a identificação de projetos multianuais.
"""

def extrair_dados_empresa_projeto(df):
    """
    Extrai CNPJ, Razão Social e Nome do Projeto das colunas concatenadas.
    """
    try:
        df_temp = df.copy()
        df_temp['cnpj_extraido'] = df_temp['empresa'].str.extract(r'CNPJ:\s*([\d./-]+)')
        df_temp['razao_social_extraida'] = df_temp['empresa'].str.extract(r'RAZÃO SOCIAL:\s*([^:]+?)(?:\s+ATIVIDADE ECONOMICA|$)')
        df_temp['nome_projeto_extraido'] = df_temp['projeto'].str.extract(r'NOME:\s*([^:]+?)(?:\s+DESCRIÇÂO|$)')
        df_temp['ciclo_multianual'] = df_temp['projeto_multianual'].str.extract(r'CICLO MAIOR QUE 1 ANO:\s*([^:]+?)(?:\s+ATIVIDADE PDI|$)')
        for col in ['cnpj_extraido', 'razao_social_extraida', 'nome_projeto_extraido', 'ciclo_multianual']:
            if col in df_temp.columns:
                df_temp[col] = df_temp[col].str.strip()
        return df_temp
    except Exception as e:
        logging.error(f"❌ Erro ao extrair dados: {e}")
        return df

def identificar_projetos_multianuais(df_temp):
    """
    Identifica grupos de projetos multianuais que devem ser ligados automaticamente.
    """
    try:
        projetos_multianuais = df_temp[
            (df_temp['ciclo_multianual'] == 'Sim') &
            (df_temp['cnpj_extraido'].notna()) &
            (df_temp['razao_social_extraida'].notna()) &
            (df_temp['nome_projeto_extraido'].notna())
        ].copy()
        
        if projetos_multianuais.empty:
            logging.info("ℹ️ Nenhum projeto multianual encontrado para agrupamento automático.")
            return []
            
        grupos = projetos_multianuais.groupby(['cnpj_extraido', 'razao_social_extraida', 'nome_projeto_extraido'])
        grupos_identificados = []
        grupo_id_counter = 1
        for _, grupo_df in grupos:
            if len(grupo_df) > 1:
                grupo_info = {
                    'grupo_id_multianual': f"MULTI_{grupo_id_counter:04d}",
                    'indices_df': grupo_df.index.tolist(),
                    'total_registros': len(grupo_df),
                    'anos': sorted(grupo_df['ano_referencia'].unique())
                }
                grupos_identificados.append(grupo_info)
                grupo_id_counter += 1
        logging.info(f"🔗 {len(grupos_identificados)} grupos de projetos multianuais identificados.")
        return grupos_identificados
    except Exception as e:
        logging.error(f"❌ Erro ao identificar projetos multianuais: {e}")
        return []

def aplicar_ligacao_automatica(df_temp, grupos_multianuais):
    """
    Aplica marcadores de grupo aos projetos multianuais identificados.
    """
    try:
        df_processado = df_temp.copy()
        df_processado['grupo_multianual'] = "0"
        df_processado['eh_multianual'] = False
        
        for grupo in grupos_multianuais:
            df_processado.loc[grupo['indices_df'], 'grupo_multianual'] = grupo['grupo_id_multianual']
            df_processado.loc[grupo['indices_df'], 'eh_multianual'] = True
            
        return df_processado
    except Exception as e:
        logging.error(f"❌ Erro ao aplicar ligação automática: {e}")
        return df_temp

def preparar_dados_com_multianual(df):
    """
    Prepara dados para a LLM, incluindo projetos multianuais para maximizar as comparações.
    """
    try:
        anos_desejados = [2020, 2021, 2022, 2023]
        df_filtrado_por_ano = df[df['ano_referencia'].isin(anos_desejados)].copy()
        
        print(f"🔍 Filtro de ano aplicado. Mantendo apenas os anos: {anos_desejados}.")
        print(f"   - Registros antes do filtro: {len(df):,}; Registros após o filtro: {len(df_filtrado_por_ano):,}")
        
        df_clean = df_filtrado_por_ano.dropna(subset=['projeto', 'setor', 'natureza', 'tipo_pesquisa', 'ano_referencia'])
        df_clean = df_clean[df_clean['projeto'].str.len() > 50].copy()
        
        df_temp = extrair_dados_empresa_projeto(df_clean)
        grupos_multianuais = identificar_projetos_multianuais(df_temp)
        df_processado = aplicar_ligacao_automatica(df_temp, grupos_multianuais)
        
        # FILTRAGEM DE MULTIANUAIS PARA A LLM
        # 1. Separa os projetos que não são multianuais
        df_nao_multianuais = df_processado[df_processado['eh_multianual'] == False].copy()
        
        # 2. Dos projetos multianuais, seleciona apenas o mais antigo de cada grupo
        df_multianuais_todos = df_processado[df_processado['eh_multianual'] == True].copy()
        if not df_multianuais_todos.empty:
            # Encontra o índice da linha com o menor 'ano_referencia' para cada 'grupo_multianual'
            idx_mais_antigos = df_multianuais_todos.loc[df_multianuais_todos.groupby('grupo_multianual')['ano_referencia'].idxmin()].index
            df_multianuais_unicos = df_multianuais_todos.loc[idx_mais_antigos]
        else:
            df_multianuais_unicos = pd.DataFrame()

        # 3. Junta os não-multianuais com os multianuais mais antigos para enviar à LLM
        df_para_llm = pd.concat([df_nao_multianuais, df_multianuais_unicos], ignore_index=True)
        
        logging.info(f"🤖 Projetos para LLM: {len(df_para_llm)} (Projetos multianuais duplicados foram removidos, mantendo apenas o mais antigo de cada grupo)")

        combinacoes = df_para_llm.groupby(['setor', 'tipo_pesquisa', 'natureza']).size().reset_index(name='count')
        combinacoes_validas = combinacoes[combinacoes['count'] >= 2].copy()
        
        if CATEGORIA_TESTE_API and isinstance(CATEGORIA_TESTE_API, str):
            combinacoes_validas = combinacoes_validas[combinacoes_validas['setor'] == CATEGORIA_TESTE_API]
            
        return df_processado, df_para_llm, combinacoes_validas, grupos_multianuais
    except Exception as e:
        logging.error(f"❌ Erro na preparação dos dados: {e}")
        return None, None, None, None

def salvar_relatorio_multianuais(grupos_multianuais):
    """
    Salva relatório detalhado dos projetos multianuais identificados.
    """
    try:
        if not grupos_multianuais: return None
        
        relatorio_data = [{'grupo_id_multianual': g['grupo_id_multianual'], 'total_registros': g['total_registros'], 'anos_projeto': ', '.join(map(str, g['anos']))} for g in grupos_multianuais]
        df_relatorio = pd.DataFrame(relatorio_data)
        
        # Garante que o diretório exista
        os.makedirs('resultados_agrupamento', exist_ok=True)
        arquivo_relatorio = 'resultados_agrupamento/projetos_multianuais_identificados.csv'
        df_relatorio.to_csv(arquivo_relatorio, index=False, encoding='utf-8', sep=';')

        logging.info(f"📄 Relatório de multianuais salvo: {arquivo_relatorio}")
        return arquivo_relatorio
    except Exception as e:
        logging.error(f"❌ Erro ao salvar relatório de multianuais: {e}")
        return None

# --- BLOCO DE EXECUÇÃO DO CHUNK 2 ---
print("\n🔄 Executando Chunk 2: Preparação dos Dados")

CATEGORIA_TESTE_API = False  # Mudar para None para processar todas as categorias

if 'df' in locals() and not df.empty:
    df_processado, df_para_llm, combinacoes_validas, grupos_multianuais = preparar_dados_com_multianual(df)

    if df_processado is not None:
        arquivo_relatorio = salvar_relatorio_multianuais(grupos_multianuais)
        
        print("\n" + "="*60)
        if CATEGORIA_TESTE_API:
            print(f"🧪 MODO TESTE ATIVADO - Categoria: '{CATEGORIA_TESTE_API}'")
            if not combinacoes_validas.empty:
                print(f"   📊 Projetos a serem processados no teste: {combinacoes_validas['count'].sum()}")
            else:
                print(f"   ⚠️ ATENÇÃO: Nenhuma combinação encontrada para a categoria de teste.")
        else:
            print("🌐 MODO COMPLETO - Processando todas as categorias")
        print("="*60)

        print(f"\n✅ Chunk 2 executado com sucesso:")
        print(f"   - Total de registros processados: {len(df_processado):,}")
        print(f"   - Projetos com marcação multianual: {df_processado['eh_multianual'].sum():,}")
        print(f"   - Total de projetos para análise da LLM: {len(df_para_llm):,}")
        print(f"   - Total de lotes (combinações) para a LLM: {len(combinacoes_validas)}")
        if grupos_multianuais:
            print(f"   - Grupos multianuais distintos: {len(grupos_multianuais)}")
        
        df_clean = df_para_llm
    else:
        print("❌ Falha na execução do Chunk 2.")
else:
    print("❌ DataFrame 'df' não encontrado. Execute o Chunk 1 primeiro.")


🔄 Executando Chunk 2: Preparação dos Dados
🔍 Filtro de ano aplicado. Mantendo apenas os anos: [2020, 2021, 2022, 2023].
   - Registros antes do filtro: 75,348; Registros após o filtro: 51,458


2025-08-18 11:07:26,470 - INFO - 🔗 4872 grupos de projetos multianuais identificados.
2025-08-18 11:07:28,219 - INFO - 🤖 Projetos para LLM: 45114 (Projetos multianuais duplicados foram removidos, mantendo apenas o mais antigo de cada grupo)
2025-08-18 11:07:28,306 - INFO - 📄 Relatório de multianuais salvo: resultados_agrupamento/projetos_multianuais_identificados.csv



🌐 MODO COMPLETO - Processando todas as categorias

✅ Chunk 2 executado com sucesso:
   - Total de registros processados: 51,446
   - Projetos com marcação multianual: 11,204
   - Total de projetos para análise da LLM: 45,114
   - Total de lotes (combinações) para a LLM: 63
   - Grupos multianuais distintos: 4872


In [5]:
"""
CHUNK 3: Configuração da API Deepseek
Configura a conexão com a API Deepseek usando LangChain
"""

def configurar_api_deepseek():
    """
    Configura o cliente da API Deepseek
    """
    try:
        # Obter chave da API
        api_key = os.getenv('DEEPSEEK_API_KEY')
        if not api_key:
            raise ValueError("A chave da API do DeepSeek não está definida nas variáveis de ambiente.")
        
        # Configurar o modelo
        model = ChatOpenAI(
            model="deepseek-chat",
            temperature=0.3,  # Baixa temperatura para resultados mais consistentes
            base_url="https://api.deepseek.com",
            api_key=api_key,
            max_tokens=4000
        )
        
        logging.info("✅ API Deepseek configurada com sucesso")
        return model
    
    except Exception as e:
        logging.error(f"❌ Erro ao configurar API Deepseek: {e}")
        return None

def estimar_custo_processamento(combinacoes_validas):
    """
    Estima o custo aproximado do processamento
    """
    if combinacoes_validas is None:
        return
    
    total_combinacoes = len(combinacoes_validas)
    total_projetos = combinacoes_validas['count'].sum()
    
    # Estimativas (valores aproximados para Deepseek)
    tokens_por_projeto = 300  # Média de tokens por projeto
    tokens_totais = total_projetos * tokens_por_projeto
    custo_por_1k_tokens = 0.0014  # USD por 1k tokens (aproximado Deepseek)
    custo_estimado = (tokens_totais / 1000) * custo_por_1k_tokens
    
    logging.info(f"💰 Estimativa de custo:")
    logging.info(f"   📊 Total de combinações: {total_combinacoes}")
    logging.info(f"   📋 Total de projetos: {total_projetos}")
    logging.info(f"   🔤 Tokens estimados: {tokens_totais:,}")
    logging.info(f"   💵 Custo estimado: ${custo_estimado:.2f} USD")
    
    return custo_estimado

# Executar configuração
print("\n🔄 Executando Chunk 3: Configuração da API Deepseek")

# Configurar API
model_deepseek = configurar_api_deepseek()

if model_deepseek:
    # Estimar custo
    custo_estimado = estimar_custo_processamento(combinacoes_validas)
        
    print(f"✅ Chunk 3 executado: API configurada e testada")
    print(f"💰 Custo estimado: ${custo_estimado:.2f} USD" if custo_estimado else "Custo não calculado")
else:
    print("❌ Chunk 3 falhou: Não foi possível configurar a API")

2025-08-18 11:08:16,268 - INFO - ✅ API Deepseek configurada com sucesso
2025-08-18 11:08:16,271 - INFO - 💰 Estimativa de custo:
2025-08-18 11:08:16,272 - INFO -    📊 Total de combinações: 63
2025-08-18 11:08:16,273 - INFO -    📋 Total de projetos: 45114
2025-08-18 11:08:16,274 - INFO -    🔤 Tokens estimados: 13,534,200
2025-08-18 11:08:16,275 - INFO -    💵 Custo estimado: $18.95 USD



🔄 Executando Chunk 3: Configuração da API Deepseek
✅ Chunk 3 executado: API configurada e testada
💰 Custo estimado: $18.95 USD


In [9]:
"""
CHUNK 4: Preparação do Template de Prompt para IA
Cria os templates de SystemMessage e HumanMessage para o Deepseek
"""

def criar_system_message():
    """
    Cria a mensagem do sistema com instruções para agrupamento
    """
    system_prompt = """Você é um especialista em análise de projetos de Pesquisa & Desenvolvimento (P&D) da Lei do Bem brasileira.

Sua tarefa é analisar projetos e agrupá-los por alta similaridade técnica e temática.

CRITÉRIOS DE AGRUPAMENTO:
1. Projetos devem ter ALTA SIMILARIDADE (>75%) em:
   - Objeto/tema principal do projeto
   - Tecnologias utilizadas
   - Metodologia aplicada
   - Resultados esperados

2. GRUPOS VÁLIDOS:
   - Mínimo: 2 projetos por grupo
   - Máximo: 8 projetos por grupo
   - Projetos únicos ficam sem grupo (grupo_id = 0)

3. CRITÉRIOS DE SIMILARIDADE:
   - Mesmo domínio tecnológico (ex: IoT, sensores, automação)
   - Mesma aplicação (ex: monitoramento, controle, otimização)
   - Metodologias similares (ex: machine learning, análise de dados)
   - Resultados comparáveis (ex: produtos, processos, softwares, serviços)

FORMATO DE SAÍDA:
Retorne APENAS um CSV, usando ';' como separador, com as colunas:
grupo_id;sublote;projeto_id;similaridade_score;justificativa_agrupamento

EXEMPLO PARA O LOTE 4 DO TRANSVERSAL - DE - PROCESSO:
TR_DE_PR_1;SL_4;ID123;0.85;"Otimização de processos industriais com foco em automação e aumento de capacidade produtiva"
TR_DE_PR_1;SL_4;ID456;0.85;"Melhorias em processos industriais incluindo robótica de solda e corte a laser"
TR_DE_PR_2;SL_4;ID789;0.88;"Sistemas de controle inteligente para operação remota de equipamentos pesados"
TR_DE_PR_2;SL_4;ID012;0.88;"Automação avançada para soluções industriais com foco em sustentabilidade"
TR_DE_PR_0;SL_4;ID345;0.00;"Projeto único sobre desenvolvimento de processos a laser"

EXEMPLO PARA O LOTE 6 DO TIC - PA - SERVIÇO:
TC_PA_SE_3;SL_6;ID234;0.88;"Solução tecnológica para módulos fiscais com foco em performance e compliance"
TC_PA_SE_3;SL_6;ID346;0.90;"Implementação de banco de dados NoSQL Redis para performance em geração de relatórios"
TC_PA_SE_0;SL_6;ID348;0.00;"Plataforma Evo para análise automatizada de crédito"

IMPORTANTE:
- Seja rigoroso na similaridade
- Prefira menos grupos com alta qualidade
- Justifique cada agrupamento brevemente
- Analise todo o contexto do projeto, não apenas palavras-chave
- Quando for falar de polegadas, não coloque aspas, escreva 'polegadas' ao invés disso
- Evite colocar aspas no texto para quando for tratar o csv, não dar erro
- Padronizar a nomenclatura dos grupos por setor, natureza e tipo:
    Transversal	= TR
    Química e Farmácia = QF
    Metalurgia e Mineração = MM
    Agroindústria e Alimentos = AA
    Eletroeletrônica = EL
    TIC = TC
    Mecânica e Transporte = MC
            
    PB - Pesquisa BásicA = PB
    PA - Pesquisa Aplicada = PA
    DE - Desenvolvimento Experimental = DE
            
    Produto = PD
    Processo = PR
    Serviço = SE
"""

    return SystemMessage(content=system_prompt)

def formatar_projetos_para_analise(df_subset):
    """
    Formata os projetos de um subset para análise pela IA
    """
    try:
        projetos_formatados = []
        
        for idx, row in df_subset.iterrows():
            # Extrair ID único do projeto da coluna 'projeto'
            projeto_texto = str(row['projeto'])
            
            # Buscar ID único entre ' ID ÚNICO: ' e ' NOME: '
            import re
            match_id = re.search(r'ID ÚNICO:\s*([^:]+?)\s+NOME:', projeto_texto)
            
            if match_id:
                projeto_id = match_id.group(1).strip()
            else:
                # Fallback caso não encontre o padrão
                projeto_id = f"PROJ_{row.get('id_empresa_ano', idx)}_{idx}"
                logging.warning(f"⚠️ ID único não encontrado para linha {idx}, usando fallback: {projeto_id}")
            
            # Formatação limpa do projeto
            projeto_formatado = f"""
ID: {projeto_id}
PROJETO: {row['projeto'][:700]}...
SETOR: {row['setor']}
NATUREZA: {row['natureza']}
TIPO: {row['tipo_pesquisa']}
RESULTADOS: {row['projeto_resultados'][:300] if pd.notna(row['projeto_resultados']) else 'Não informado'}...
"""
            projetos_formatados.append(projeto_formatado.strip())
        
        logging.info(f"📋 Formatados {len(projetos_formatados)} projetos para análise")
        return projetos_formatados
    
    except Exception as e:
        logging.error(f"❌ Erro ao formatar projetos: {e}")
        return []


def criar_human_message(projetos_formatados, combinacao_info, df_subset, sublote_num):
    """
    Cria a mensagem humana com os projetos para análise, incluindo o número do sublote.
    """
    try:
        anos_presentes = sorted(df_subset['ano_referencia'].unique())
        anos_str = '; '.join(map(str, [int(ano) for ano in anos_presentes]))

        # Cabeçalho da análise atualizado para incluir o sublote
        cabecalho = f"""Analise os projetos abaixo e agrupe-os por alta similaridade técnica.

CONTEXTO DA ANÁLISE:
- Setor: {combinacao_info['setor']}
- Tipo de Pesquisa: {combinacao_info['tipo_pesquisa']}
- Natureza: {combinacao_info['natureza']}
- Anos neste lote: {anos_str}
- Total de Projetos: {len(projetos_formatados)}
- SUBLOTE ATUAL: SL_{sublote_num}

PROJETOS PARA ANÁLISE:
{'='*50}"""

        projetos_texto = '\n\n'.join(projetos_formatados)
        
        instrucao_final = f"""
{'='*50}

Retorne APENAS o CSV com o agrupamento, seguindo o formato especificado no system prompt.
Lembre-se de preencher a coluna 'sublote' com o valor 'SL_{sublote_num}' para todas as linhas.
Analise cuidadosamente a similaridade técnica entre os projetos."""

        mensagem_completa = f"{cabecalho}\n\n{projetos_texto}\n{instrucao_final}"
        
        return HumanMessage(content=mensagem_completa)
    
    except Exception as e:
        logging.error(f"❌ Erro ao criar human message: {e}")
        return None

def validar_tamanho_prompt(system_msg, human_msg, limite_tokens=50000):
    """
    Valida se o prompt não excede o limite de tokens
    """
    try:
        # Estimativa simples: ~4 caracteres por token
        total_chars = len(system_msg.content) + len(human_msg.content)
        tokens_estimados = total_chars // 4
        
        if tokens_estimados > limite_tokens:
            logging.warning(f"⚠️ Prompt muito longo: {tokens_estimados} tokens estimados")
            return False
        
        logging.info(f"✅ Tamanho do prompt OK: {tokens_estimados} tokens estimados")
        return True
    
    except Exception as e:
        logging.error(f"❌ Erro ao validar tamanho do prompt: {e}")
        return False

print("\n🔄 Executando Chunk 4: Preparação do Template de Prompt")

# Criar system message (você já tem esta função, apenas garanta que a coluna 'sublote' está no header do CSV)
system_message_template = criar_system_message()

# Teste com dados dummy (atualizado para incluir sublote_num)
if 'df_clean' in locals() and not df_clean.empty:
    df_teste = df_clean.head(3)
    projetos_teste = formatar_projetos_para_analise(df_teste)
    
    combinacao_teste = {
        'setor': df_teste.iloc[0]['setor'],
        'tipo_pesquisa': df_teste.iloc[0]['tipo_pesquisa'],
        'natureza': df_teste.iloc[0]['natureza']
    }
    
    # Passa um número de sublote de exemplo (99) para o teste
    human_message_teste = criar_human_message(projetos_teste, combinacao_teste, df_teste, sublote_num=99)
    
    if human_message_teste:
        prompt_valido = validar_tamanho_prompt(system_message_template, human_message_teste)
        print(f"✅ Chunk 4 executado: Template criado e validado")
    else:
        print("⚠️ Chunk 4 parcial: Template de teste falhou")
else:
    print("✅ Chunk 4 executado: Template criado (sem dados para teste)")


2025-08-18 11:46:12,664 - INFO - 📋 Formatados 3 projetos para análise
2025-08-18 11:46:12,667 - INFO - ✅ Tamanho do prompt OK: 1675 tokens estimados



🔄 Executando Chunk 4: Preparação do Template de Prompt
✅ Chunk 4 executado: Template criado e validado


In [10]:
"""
CHUNK 5: Processamento por Categoria com Sub-lotes Inteligentes
Organiza processamento garantindo que todos os projetos de uma categoria sejam comparados
"""

# Novo limite baseado em tokens, não quantidade arbitrária
LIMITE_TOKENS_SEGUROS = 50000  # Deixa margem para resposta
TOKENS_POR_PROJETO = 350      # Estimativa conservadora
LIMITE_PROJETOS_POR_SUBLOTE = int(LIMITE_TOKENS_SEGUROS / TOKENS_POR_PROJETO)  # ~142 projetos

def dividir_categoria_em_sublotes(df_categoria, limite_projetos=LIMITE_PROJETOS_POR_SUBLOTE):
    """
    Divide uma categoria grande em sub-lotes, mantendo sobreposição para merge posterior
    """
    try:
        total_projetos = len(df_categoria)
        
        if total_projetos <= limite_projetos:
            # Categoria pequena: processar tudo de uma vez
            return [{
                'dados': df_categoria,
                'sublote_num': 1,
                'total_sublotes': 1,
                'tipo': 'categoria_completa',
                'sobreposicao': None
            }]
        
        # Categoria grande: dividir com sobreposição para merge posterior
        sublotes = []
        overlap_size = min(20, limite_projetos // 4)  # 25% de sobreposição, máximo 20

        inicio = 0
        sublote_num = 1
        
        while inicio < total_projetos:
            fim = min(inicio + limite_projetos, total_projetos)
            
            # Adicionar sobreposição (exceto no primeiro sub-lote)
            if sublote_num > 1:
                inicio_real = max(0, inicio - overlap_size)
            else:
                inicio_real = inicio
            
            sublote_dados = df_categoria.iloc[inicio_real:fim]
            
            sublote = {
                'dados': sublote_dados,
                'sublote_num': sublote_num,
                'total_sublotes': None,  # Será calculado depois
                'tipo': 'sublote_categoria',
                'sobreposicao': {
                    'inicio_original': inicio,
                    'fim_original': fim,
                    'overlap_inicio': overlap_size if sublote_num > 1 else 0,
                    'projetos_sobrepostos': overlap_size if sublote_num > 1 else 0
                }
            }
            sublotes.append(sublote)
            
            inicio = fim
            sublote_num += 1
        
        # Atualizar total de sub-lotes
        for sublote in sublotes:
            sublote['total_sublotes'] = len(sublotes)
        
        logging.info(f"📦 Categoria dividida: {total_projetos} projetos → {len(sublotes)} sub-lotes")
        logging.info(f"   🔗 Sobreposição: {overlap_size} projetos entre sub-lotes")
        
        return sublotes
    
    except Exception as e:
        logging.error(f"❌ Erro ao dividir categoria: {e}")
        return []

def filtrar_categoria_especifica(df_clean, combinacao):
    """
    Filtra TODOS os projetos de uma categoria específica (incluindo anos diferentes)
    """
    try:
        df_categoria = df_clean[
            (df_clean['setor'] == combinacao['setor']) &
            (df_clean['tipo_pesquisa'] == combinacao['tipo_pesquisa']) &
            (df_clean['natureza'] == combinacao['natureza'])
        ].copy()
        
        # Adicionar informação de ano para controle
        anos_encontrados = df_categoria['ano_referencia'].unique()
        
        logging.info(f"🔍 Categoria filtrada com Tipo '{combinacao['tipo_pesquisa']}' e Natureza '{combinacao['natureza']}': {len(df_categoria)} projetos")
        logging.info(f"📅 Anos encontrados: {sorted(anos_encontrados)}")
        
        return df_categoria
    
    except Exception as e:
        logging.error(f"❌ Erro ao filtrar categoria: {e}")
        return pd.DataFrame()

def criar_nome_arquivo_categoria(combinacao, sublote_info=None):
    """
    Cria nome padronizado para arquivos por categoria
    """
    try:
        # Limpar caracteres especiais
        setor = re.sub(r'[^\w\s-]', '', str(combinacao['setor'])).strip()[:25]
        tipo = re.sub(r'[^\w\s-]', '', str(combinacao['tipo_pesquisa'])).strip()[:25]
        natureza = re.sub(r'[^\w\s-]', '', str(combinacao['natureza'])).strip()[:20]
        
        # Substituir espaços por underscores
        setor = setor.replace(' ', '_')
        tipo = tipo.replace(' ', '_')
        natureza = natureza.replace(' ', '_')
        
        if sublote_info and sublote_info['tipo'] == 'sublote_categoria':
            nome = f"grupos_categoria_{setor}_{tipo}_{natureza}_sublote{sublote_info['sublote_num']}.csv"
        else:
            nome = f"grupos_categoria_{setor}_{tipo}_{natureza}_completa.csv"
        
        return nome
    
    except Exception as e:
        logging.error(f"❌ Erro ao criar nome do arquivo: {e}")
        return f"grupos_categoria_erro_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

def preparar_plano_processamento_categoria(df_clean, combinacoes_validas):
    """
    Prepara plano de processamento focado em categorias completas
    """
    try:
        plano_processamento = []
        
        for idx, row in combinacoes_validas.iterrows():
            combinacao = {
                'setor': row['setor'],
                'tipo_pesquisa': row['tipo_pesquisa'],
                'natureza': row['natureza'],
                'count': row['count']
            }
            
            # Filtrar TODOS os projetos da categoria
            df_categoria = filtrar_categoria_especifica(df_clean, combinacao)
            
            if len(df_categoria) == 0:
                continue
            
            # Dividir em sub-lotes se necessário
            sublotes = dividir_categoria_em_sublotes(df_categoria)
            
            for sublote in sublotes:
                plano_item = {
                    'combinacao': combinacao,
                    'dados': sublote['dados'],
                    'sublote_info': sublote,
                    'arquivo_saida': criar_nome_arquivo_categoria(combinacao, sublote),
                    'requer_merge': sublote['tipo'] == 'sublote_categoria'
                }
                plano_processamento.append(plano_item)
        
        logging.info(f"📋 Plano por categoria criado: {len(plano_processamento)} itens")
        
        # Estatísticas do plano
        categorias_completas = sum(1 for item in plano_processamento if not item['requer_merge'])
        categorias_sublotes = len(plano_processamento) - categorias_completas
        
        logging.info(f"   🟢 Categorias processadas completas: {categorias_completas}")
        logging.info(f"   🟡 Sub-lotes para merge posterior: {categorias_sublotes}")
        
        return plano_processamento
    
    except Exception as e:
        logging.error(f"❌ Erro ao preparar plano por categoria: {e}")
        return []

def salvar_plano_categoria(plano_processamento):
    """
    Salva o plano de processamento por categoria
    """
    try:
        plano_resumo = []
        for item in plano_processamento:
            resumo = {
                'setor': item['combinacao']['setor'],
                'tipo_pesquisa': item['combinacao']['tipo_pesquisa'],
                'natureza': item['combinacao']['natureza'],
                'total_projetos': len(item['dados']),
                'sublote_num': item['sublote_info']['sublote_num'],
                'total_sublotes': item['sublote_info']['total_sublotes'],
                'tipo_processamento': item['sublote_info']['tipo'],
                'requer_merge': item['requer_merge'],
                'arquivo_saida': item['arquivo_saida']
            }
            plano_resumo.append(resumo)
        
        df_plano = pd.DataFrame(plano_resumo)
        arquivo_plano = 'resultados_agrupamento/plano_processamento_categoria.csv'
        df_plano.to_csv(arquivo_plano, index=False, encoding='utf-8', sep=';')
        
        logging.info(f"📄 Plano por categoria salvo: {arquivo_plano}")
        return arquivo_plano
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar plano categoria: {e}")
        return None

# Executar preparação por categoria
print("\n🔄 Executando Chunk 5 ADAPTADO: Processamento por Categoria")

if 'df_clean' in locals() and 'combinacoes_validas' in locals() and df_clean is not None:
    print(f"🎯 Nova estratégia: Categorias completas com sub-lotes inteligentes")
    print(f"📊 Limite por sub-lote: {LIMITE_PROJETOS_POR_SUBLOTE} projetos (~{LIMITE_TOKENS_SEGUROS:,} tokens)")
    
    # Criar plano por categoria
    plano_processamento = preparar_plano_processamento_categoria(df_clean, combinacoes_validas)
    
    if plano_processamento:
        # Salvar plano
        arquivo_plano = salvar_plano_categoria(plano_processamento)
        
        # Estatísticas finais
        total_projetos = sum(len(item['dados']) for item in plano_processamento)
        categorias_unicas = len(set((item['combinacao']['setor'], 
                                   item['combinacao']['tipo_pesquisa'], 
                                   item['combinacao']['natureza']) 
                                  for item in plano_processamento))
        
        print(f"✅ Chunk 5 adaptado executado:")
        print(f"   📊 Total de itens para processar: {len(plano_processamento)}")
        print(f"   🏷️ Categorias únicas: {categorias_unicas}")
        print(f"   📋 Total de projetos: {total_projetos}")
        print(f"   📄 Plano salvo: {arquivo_plano}")
        print(f"   🔗 Estratégia: Comparação completa dentro de cada categoria")
    else:
        print("❌ Chunk 5 adaptado falhou: Não foi possível criar plano de categoria")
else:
    print("⚠️ Chunk 5 adaptado ignorado: Dados não disponíveis dos chunks anteriores")

2025-08-18 11:46:18,015 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Processo': 1425 projetos
2025-08-18 11:46:18,016 - INFO - 📅 Anos encontrados: [np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-18 11:46:18,017 - INFO - 📦 Categoria dividida: 1425 projetos → 11 sub-lotes
2025-08-18 11:46:18,018 - INFO -    🔗 Sobreposição: 20 projetos entre sub-lotes
2025-08-18 11:46:18,052 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Produto': 2595 projetos
2025-08-18 11:46:18,053 - INFO - 📅 Anos encontrados: [np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-18 11:46:18,054 - INFO - 📦 Categoria dividida: 2595 projetos → 19 sub-lotes
2025-08-18 11:46:18,055 - INFO -    🔗 Sobreposição: 20 projetos entre sub-lotes
2025-08-18 11:46:18,067 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Serviço': 113 projetos


🔄 Executando Chunk 5 ADAPTADO: Processamento por Categoria
🎯 Nova estratégia: Categorias completas com sub-lotes inteligentes
📊 Limite por sub-lote: 142 projetos (~50,000 tokens)


2025-08-18 11:46:18,169 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Processo': 844 projetos
2025-08-18 11:46:18,169 - INFO - 📅 Anos encontrados: [np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-18 11:46:18,170 - INFO - 📦 Categoria dividida: 844 projetos → 6 sub-lotes
2025-08-18 11:46:18,171 - INFO -    🔗 Sobreposição: 20 projetos entre sub-lotes
2025-08-18 11:46:18,195 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Produto': 2005 projetos
2025-08-18 11:46:18,196 - INFO - 📅 Anos encontrados: [np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)]
2025-08-18 11:46:18,197 - INFO - 📦 Categoria dividida: 2005 projetos → 15 sub-lotes
2025-08-18 11:46:18,197 - INFO -    🔗 Sobreposição: 20 projetos entre sub-lotes
2025-08-18 11:46:18,209 - INFO - 🔍 Categoria filtrada com Tipo 'DE - Desenvolvimento Experimental' e Natureza 'Serviço': 95 projetos
202

✅ Chunk 5 adaptado executado:
   📊 Total de itens para processar: 352
   🏷️ Categorias únicas: 63
   📋 Total de projetos: 50894
   📄 Plano salvo: resultados_agrupamento/plano_processamento_categoria.csv
   🔗 Estratégia: Comparação completa dentro de cada categoria


In [11]:
"""
CHUNK 6: Loop Principal de Iteração com Requisições Assíncronas
Implementa processamento assíncrono para acelerar requisições à API
"""

import asyncio
import concurrent.futures
from functools import partial

MAX_CONCURRENT_REQUESTS = 20  # Limite de requisições simultâneas
TAMANHO_LOTE = 20

def configurar_api_deepseek_async():
    """
    Configura o cliente da API Deepseek (usando ChatOpenAI normal)
    """
    try:
        # Obter chave da API
        api_key = os.getenv('DEEPSEEK_API_KEY')
        if not api_key:
            raise ValueError("A chave da API do DeepSeek não está definida nas variáveis de ambiente.")
        
        # Configurar o modelo (usar o ChatOpenAI normal)
        model = ChatOpenAI(
            model="deepseek-chat",
            temperature=0.3,
            base_url="https://api.deepseek.com",
            api_key=api_key,
            max_tokens=4000,
            max_retries=3,
            request_timeout=60.0
        )
        
        logging.info("✅ API Deepseek configurada para uso assíncrono")
        return model
    
    except Exception as e:
        logging.error(f"❌ Erro ao configurar API Deepseek: {e}")
        return None

async def processar_item_plano_async(item_plano, model_deepseek, system_message_template, executor, semaforo):
    """
    Processa um item individual do plano, extraindo e passando o número do sublote.
    """
    async with semaforo:
        try:
            combinacao = item_plano['combinacao']
            dados = item_plano['dados']
            arquivo_saida = item_plano['arquivo_saida']
            # *** NOVA LINHA: Extrai o número do sublote ***
            sublote_num = item_plano['sublote_info']['sublote_num']
            
            logging.info(f"🔄 Processando async: {combinacao['setor']} - Sublote {sublote_num} ({len(dados)} projetos)")
            
            projetos_formatados = formatar_projetos_para_analise(dados)
            if not projetos_formatados:
                return None
            
            # *** ATUALIZAÇÃO: Passa 'sublote_num' como novo argumento ***
            human_message = criar_human_message(projetos_formatados, combinacao, dados, sublote_num)
            
            if not human_message:
                return None
            if not validar_tamanho_prompt(system_message_template, human_message):
                return None
            
            mensagens = [system_message_template, human_message]
            logging.info(f"📤 Enviando para API Deepseek (Sublote {sublote_num})...")
            
            loop = asyncio.get_event_loop()
            resposta = await loop.run_in_executor(
                executor, 
                partial(model_deepseek.invoke, mensagens)
            )
            
            if resposta and resposta.content:
                logging.info(f"✅ Resposta recebida para o Sublote {sublote_num}")
                return {
                    'resposta': resposta.content,
                    'combinacao': combinacao,
                    'arquivo_saida': arquivo_saida,
                    'total_projetos': len(dados),
                    'requer_merge': item_plano.get('requer_merge', False)
                }
            else:
                return None
        except Exception as e:
            logging.error(f"❌ Erro ao processar item async: {e}")
            return None

async def executar_lote_async(lote_itens, model_deepseek, system_message_template, executor, max_concurrent=10):
    """
    Executa um lote de itens de forma assíncrona
    """
    try:
        # Semáforo para limitar requisições simultâneas
        semaforo = asyncio.Semaphore(max_concurrent)
        
        # Criar tarefas assíncronas
        tarefas = []
        for item in lote_itens:
            tarefa = processar_item_plano_async(item, model_deepseek, system_message_template, executor, semaforo)
            tarefas.append(tarefa)
        
        # Executar todas as tarefas em paralelo
        resultados = await asyncio.gather(*tarefas, return_exceptions=True)
        
        # Filtrar resultados válidos
        resultados_validos = []
        for resultado in resultados:
            if isinstance(resultado, Exception):
                logging.error(f"❌ Exceção no processamento assíncrono: {resultado}")
            elif resultado is not None:
                resultados_validos.append(resultado)
        
        logging.info(f"✅ Lote assíncrono concluído: {len(resultados_validos)}/{len(lote_itens)} sucessos")
        return resultados_validos
    
    except Exception as e:
        logging.error(f"❌ Erro no lote assíncrono: {e}")
        return []

def dividir_em_lotes_async(plano_processamento, tamanho_lote=10):
    """
    Divide o plano em lotes menores para processamento assíncrono
    """
    try:
        lotes = []
        total_itens = len(plano_processamento)
        
        for i in range(0, total_itens, tamanho_lote):
            lote = plano_processamento[i:i+tamanho_lote]
            lotes.append(lote)
        
        logging.info(f"📦 Divisão para processamento assíncrono: {total_itens} itens → {len(lotes)} lotes")
        return lotes
    
    except Exception as e:
        logging.error(f"❌ Erro ao dividir em lotes async: {e}")
        return [plano_processamento]  # Retorna como um lote único

async def executar_loop_principal_async(plano_processamento, model_deepseek, system_message_template, 
                                       modo_teste=False, limite_teste=5, max_concurrent=MAX_CONCURRENT_REQUESTS, tamanho_lote=TAMANHO_LOTE):
    """
    Executa o loop principal de processamento de forma assíncrona
    """
    try:
        resultados = []
        total_itens = len(plano_processamento)
        
        # Limitar para teste se necessário
        if modo_teste:
            plano_processamento = plano_processamento[:limite_teste]
            logging.info(f"🧪 Modo teste ativado: processando apenas {len(plano_processamento)} itens")
        
        logging.info(f"🚀 Iniciando processamento assíncrono de {len(plano_processamento)} itens...")
        logging.info(f"⚡ Configuração: {max_concurrent} requisições simultâneas, lotes de {tamanho_lote}")
        
        # Criar ThreadPoolExecutor para requisições HTTP
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent) as executor:
            # Dividir em lotes para gerenciar melhor o processamento
            lotes = dividir_em_lotes_async(plano_processamento, tamanho_lote=TAMANHO_LOTE)

            for idx_lote, lote in enumerate(lotes, 1):
                try:
                    logging.info(f"\n{'='*60}")
                    logging.info(f"📦 Processando lote {idx_lote}/{len(lotes)} ({len(lote)} itens)")
                    
                    # Executar lote assíncrono
                    resultados_lote = await executar_lote_async(
                        lote, model_deepseek, system_message_template, executor, max_concurrent
                    )
                    
                    resultados.extend(resultados_lote)
                    
                    logging.info(f"✅ Lote {idx_lote} concluído: {len(resultados_lote)} sucessos")
                    
                    # Pausa entre lotes (exceto no último)
                    if idx_lote < len(lotes):
                        pausa_entre_lotes = min(TEMPO_PAUSA_ENTRE_REQUESTS, 3)  # Máximo 3s entre lotes
                        logging.info(f"⏸️ Pausando {pausa_entre_lotes}s entre lotes...")
                        await asyncio.sleep(pausa_entre_lotes)
                    
                except Exception as e:
                    logging.error(f"❌ Erro no lote {idx_lote}: {e}")
                    continue
        
        logging.info(f"\n🎉 Processamento assíncrono concluído!")
        logging.info(f"✅ Sucessos: {len(resultados)}/{total_itens}")
        
        return resultados
    
    except Exception as e:
        logging.error(f"❌ Erro no loop principal assíncrono: {e}")
        return []

def executar_processamento_sincronizado(plano_processamento, system_message_template, 
                                      modo_teste=True, limite_teste=5):
    """
    Função wrapper que gerencia a execução assíncrona usando ThreadPoolExecutor
    """
    try:
        # Configurar cliente (ChatOpenAI normal)
        model_deepseek = configurar_api_deepseek_async()
        if not model_deepseek:
            raise Exception("Falha na configuração da API")
        
        # Verificar se já há um loop de eventos rodando
        try:
            loop = asyncio.get_running_loop()
            logging.info("📡 Loop de eventos já ativo, usando nest_asyncio")
            
            # Se já há um loop, usar nest_asyncio
            try:
                import nest_asyncio
                nest_asyncio.apply()
            except ImportError:
                logging.warning("⚠️ nest_asyncio não instalado, tentando execução alternativa")
                # Fallback: executar de forma síncrona
                return executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste)
            
            # Executar de forma assíncrona no loop existente
            resultado = asyncio.run(executar_loop_principal_async(
                plano_processamento, model_deepseek, system_message_template,
                modo_teste, limite_teste, max_concurrent=MAX_CONCURRENT_REQUESTS, tamanho_lote=TAMANHO_LOTE
            ))
            
        except RuntimeError:
            # Não há loop rodando, criar um novo
            logging.info("📡 Criando novo loop de eventos")
            resultado = asyncio.run(executar_loop_principal_async(
                plano_processamento, model_deepseek, system_message_template,
                modo_teste, limite_teste, max_concurrent=MAX_CONCURRENT_REQUESTS, tamanho_lote=TAMANHO_LOTE
            ))
        
        return resultado
    
    except Exception as e:
        logging.error(f"❌ Erro na execução sincronizada: {e}")
        # Fallback para processamento síncrono
        return executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste)

def executar_fallback_sincronizado(plano_processamento, model_deepseek, system_message_template, modo_teste, limite_teste):
    """
    Fallback para processamento síncrono caso asyncio falhe
    """
    try:
        logging.info("🔄 Executando fallback síncrono...")
        resultados = []
        
        # Limitar para teste se necessário
        if modo_teste:
            plano_processamento = plano_processamento[:limite_teste]
        
        for idx, item in enumerate(plano_processamento, 1):
            try:
                logging.info(f"📋 Processando item {idx}/{len(plano_processamento)} (síncrono)")
                
                # Processar item de forma síncrona
                combinacao = item['combinacao']
                dados = item['dados']
                arquivo_saida = item['arquivo_saida']
                
                projetos_formatados = formatar_projetos_para_analise(dados)
                if not projetos_formatados:
                    continue
                
                human_message = criar_human_message(projetos_formatados, combinacao, dados)
                if not human_message:
                    continue
                
                mensagens = [system_message_template, human_message]
                resposta = model_deepseek.invoke(mensagens)
                
                if resposta and resposta.content:
                    resultado = {
                        'resposta': resposta.content,
                        'combinacao': combinacao,
                        'arquivo_saida': arquivo_saida,
                        'total_projetos': len(dados),
                        'requer_merge': item.get('requer_merge', False)
                    }
                    resultados.append(resultado)
                    logging.info(f"✅ Item {idx} processado (síncrono)")
                
                # Pausa entre requisições
                if idx < len(plano_processamento):
                    time.sleep(TEMPO_PAUSA_ENTRE_REQUESTS)
                
            except Exception as e:
                logging.error(f"❌ Erro no item {idx}: {e}")
                continue
        
        return resultados
    
    except Exception as e:
        logging.error(f"❌ Erro no fallback síncrono: {e}")
        return []

def salvar_progresso_intermediario(resultados):
    """
    Salva progresso intermediário durante o processamento
    """
    try:
        arquivo_progresso = f'resultados_agrupamento/progresso_intermediario.json'
        
        # Converter para formato serializável
        progresso_data = []
        for resultado in resultados:
            item = {
                'combinacao': resultado['combinacao'],
                'arquivo_saida': resultado['arquivo_saida'],
                'total_projetos': resultado['total_projetos'],
                'resposta_length': len(resultado['resposta']),
                'requer_merge': resultado.get('requer_merge', False),
                'timestamp': datetime.now().isoformat()
            }
            progresso_data.append(item)
        
        with open(arquivo_progresso, 'w', encoding='utf-8') as f:
            json.dump(progresso_data, f, indent=2, ensure_ascii=False)
        
        logging.info(f"💾 Progresso salvo: {arquivo_progresso}")
        return arquivo_progresso
    
    except Exception as e:
        logging.error(f"❌ Erro ao salvar progresso: {e}")
        return None

# Executar loop principal assíncrono
print("\n🔄 Executando Chunk 6: Loop Principal com Processamento Assíncrono")

try:
    # Acessa diretamente as variáveis. Se não existirem, um NameError será gerado.
    if not plano_processamento:
        print("✅ Chunk 6 executado: Plano de processamento está vazio. Nada a fazer.")
        resultados_processamento = [] # Garante que a variável exista para os próximos chunks
    else:
        # Timestamp para controle
        timestamp_execucao = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        print("⚡ PROCESSAMENTO ASSÍNCRONO ATIVADO")
        print(f"🧪 Executando em MODO TESTE ({min(3, len(plano_processamento))} primeiros itens)")
        print("📊 Configuração: 3 requisições simultâneas, lotes de 5 itens")
        print("Para executar completo, mude modo_teste=False")
        
        # Executar processamento assíncrono
        resultados_processamento = executar_processamento_sincronizado(
            plano_processamento, 
            system_message_template,
            modo_teste=False,  # Mudar para False para execução completa
            limite_teste=5
        )
        
        if resultados_processamento:
            # Salvar progresso
            arquivo_progresso = salvar_progresso_intermediario(resultados_processamento)
            
            print(f"✅ Chunk 6 adaptado executado: {len(resultados_processamento)} itens processados")
            print(f"⚡ Vantagem assíncrona: ~3x mais rápido que processamento sequencial")
            print(f"💾 Progresso salvo: {arquivo_progresso}")
        else:
            print("❌ Chunk 6 adaptado falhou: Nenhum item foi processado com sucesso")

except NameError as e:
    print(f"❌ Chunk 6 adaptado ignorado: Dependência não encontrada - {e}")
    print("👉 Certifique-se de que os Chunks 4 e 5 foram executados com sucesso antes deste.")

2025-08-18 11:46:42,434 - INFO - ✅ API Deepseek configurada para uso assíncrono
2025-08-18 11:46:42,436 - INFO - 📡 Loop de eventos já ativo, usando nest_asyncio
2025-08-18 11:46:42,450 - INFO - 🚀 Iniciando processamento assíncrono de 352 itens...
2025-08-18 11:46:42,451 - INFO - ⚡ Configuração: 20 requisições simultâneas, lotes de 20
2025-08-18 11:46:42,452 - INFO - 📦 Divisão para processamento assíncrono: 352 itens → 18 lotes
2025-08-18 11:46:42,452 - INFO - 
2025-08-18 11:46:42,453 - INFO - 📦 Processando lote 1/18 (20 itens)
2025-08-18 11:46:42,454 - INFO - 🔄 Processando async: Agroindústria e Alimentos - Sublote 1 (142 projetos)
2025-08-18 11:46:42,460 - INFO - 📋 Formatados 142 projetos para análise
2025-08-18 11:46:42,462 - INFO - ✅ Tamanho do prompt OK: 39292 tokens estimados
2025-08-18 11:46:42,463 - INFO - 📤 Enviando para API Deepseek (Sublote 1)...
2025-08-18 11:46:42,466 - INFO - 🔄 Processando async: Agroindústria e Alimentos - Sublote 2 (162 projetos)
2025-08-18 11:46:42,473 


🔄 Executando Chunk 6: Loop Principal com Processamento Assíncrono
⚡ PROCESSAMENTO ASSÍNCRONO ATIVADO
🧪 Executando em MODO TESTE (3 primeiros itens)
📊 Configuração: 3 requisições simultâneas, lotes de 5 itens
Para executar completo, mude modo_teste=False


2025-08-18 11:46:42,579 - INFO - 📋 Formatados 162 projetos para análise
2025-08-18 11:46:42,580 - INFO - ✅ Tamanho do prompt OK: 37838 tokens estimados
2025-08-18 11:46:42,581 - INFO - 📤 Enviando para API Deepseek (Sublote 9)...
2025-08-18 11:46:42,583 - INFO - 🔄 Processando async: Agroindústria e Alimentos - Sublote 10 (162 projetos)
2025-08-18 11:46:42,593 - INFO - 📋 Formatados 162 projetos para análise
2025-08-18 11:46:42,594 - INFO - ✅ Tamanho do prompt OK: 37056 tokens estimados
2025-08-18 11:46:42,595 - INFO - 📤 Enviando para API Deepseek (Sublote 10)...
2025-08-18 11:46:42,598 - INFO - 🔄 Processando async: Agroindústria e Alimentos - Sublote 11 (25 projetos)
2025-08-18 11:46:42,602 - INFO - 📋 Formatados 25 projetos para análise
2025-08-18 11:46:42,603 - INFO - ✅ Tamanho do prompt OK: 6594 tokens estimados
2025-08-18 11:46:42,604 - INFO - 📤 Enviando para API Deepseek (Sublote 11)...
2025-08-18 11:46:42,607 - INFO - 🔄 Processando async: Agroindústria e Alimentos - Sublote 1 (142 p

✅ Chunk 6 adaptado executado: 351 itens processados
⚡ Vantagem assíncrona: ~3x mais rápido que processamento sequencial
💾 Progresso salvo: resultados_agrupamento/progresso_intermediario.json


In [21]:
for i in range(len(plano_processamento)):
    if i < 5:
        print(plano_processamento[i])

print(system_message_template)

{'combinacao': {'setor': 'Química e Farmácia', 'tipo_pesquisa': 'DE - Desenvolvimento Experimental', 'natureza': 'Processo', 'count': 1504}, 'dados':       id_empresa_ano  ano_referencia               setor  natureza  \
76            6702.0          2021.0  Química e Farmácia  Processo   
392          25509.0          2023.0  Química e Farmácia  Processo   
1003         26910.0          2023.0  Química e Farmácia  Processo   
1036         27386.0          2023.0  Química e Farmácia  Processo   
1094         28659.0          2023.0  Química e Farmácia  Processo   
...              ...             ...                 ...       ...   
4320         28377.0          2023.0  Química e Farmácia  Processo   
4469         26564.0          2023.0  Química e Farmácia  Processo   
4495         25652.0          2023.0  Química e Farmácia  Processo   
4533         28257.0          2023.0  Química e Farmácia  Processo   
4539         28022.0          2023.0  Química e Farmácia  Processo   

         

In [12]:
"""
CHUNK 7: Validação, Limpeza e Salvamento dos Resultados
Valida a resposta da API, limpa para extrair o CSV e salva os arquivos.
"""

# ADICIONADO: Função para salvar os arquivos CSV validados
def salvar_resultados_validados(resultados_validos):
    """
    Salva os resultados de CSV limpos em seus respectivos arquivos de saída.
    """
    arquivos_salvos = []
    if not resultados_validos:
        logging.warning("Nenhum resultado válido para salvar.")
        return arquivos_salvos
        
    diretorio_saida = 'resultados_agrupamento'
    
    for resultado in resultados_validos:
        try:
            if 'arquivo_saida' not in resultado or not resultado['arquivo_saida']:
                logging.error("Resultado sem 'arquivo_saida'. Pulando.")
                continue

            caminho_arquivo = os.path.join(diretorio_saida, resultado['arquivo_saida'])
            os.makedirs(os.path.dirname(caminho_arquivo), exist_ok=True)

            with open(caminho_arquivo, 'w', encoding='utf-8') as f:
                f.write(resultado['resposta_limpa'])
            
            logging.info(f"✅ Arquivo CSV salvo: {caminho_arquivo}")
            arquivos_salvos.append(caminho_arquivo)

        except Exception as e:
            logging.error(f"❌ Erro ao salvar o arquivo {resultado.get('arquivo_saida', 'N/A')}: {e}")
            
    return arquivos_salvos

### --- FUNÇÕES AUXILIARES DO CHUNK --- ###

def salvar_resultado_invalido_txt(resultado_invalido):
    """
    Salva a resposta bruta de um resultado inválido como um arquivo .txt em uma pasta separada.
    """
    try:
        if 'arquivo_saida' not in resultado_invalido or not resultado_invalido['arquivo_saida']:
            logging.error("Resultado inválido sem 'arquivo_saida'. Pulando salvamento de .txt.")
            return None

        # Troca a extensão de .csv para .txt
        nome_arquivo_base, _ = os.path.splitext(resultado_invalido['arquivo_saida'])
        arquivo_txt = nome_arquivo_base + '.txt'
        
        # Salva em uma subpasta para manter a organização
        diretorio_saida = 'resultados_agrupamento/respostas_invalidas'
        caminho_arquivo = os.path.join(diretorio_saida, arquivo_txt)
        
        os.makedirs(os.path.dirname(caminho_arquivo), exist_ok=True)

        # Salva o conteúdo bruto da resposta
        with open(caminho_arquivo, 'w', encoding='utf-8') as f:
            f.write("--- RESPOSTA BRUTA DA API ---\n\n")
            f.write(resultado_invalido.get('resposta', 'Nenhuma resposta encontrada no objeto.'))
        
        logging.info(f"💾 Resposta inválida salva em: {caminho_arquivo}")
        return caminho_arquivo

    except Exception as e:
        logging.error(f"❌ Erro ao salvar o arquivo .txt para {resultado_invalido.get('arquivo_saida', 'N/A')}: {e}")
        return None

# Função para processar item novamente, passando o bloco novamente para a LLM
async def processar_item_com_retry(item_plano, model_deepseek, system_message_template, max_tentativas=MAX_TENTATIVAS):
    for tentativa in range(1, max_tentativas + 1):
        try:
            logging.info(f"🔄 Tentativa {tentativa}/{max_tentativas}")
            resultado = await processar_item_plano_async(item_plano, model_deepseek, system_message_template)
            if resultado:
                if tentativa > 1:
                    logging.info(f"✅ Sucesso na tentativa {tentativa}")
                return resultado
            else:
                logging.warning(f"⚠️ Tentativa {tentativa} falhou - resultado vazio")
        except Exception as e:
            logging.error(f"❌ Tentativa {tentativa} falhou: {e}")
            if tentativa < max_tentativas:
                pausa = TEMPO_PAUSA_ENTRE_REQUESTS * (2 ** (tentativa - 1))
                logging.info(f"⏸️ Pausando {pausa}s antes da próxima tentativa...")
                time.sleep(pausa)
    logging.error(f"❌ Todas as {max_tentativas} tentativas falharam")
    return None

# Função para validar a resposta da LLM
def validar_resposta_api(resposta_content):
    try:
        if not resposta_content or not isinstance(resposta_content, str):
            return False, "Conteúdo da resposta está vazio ou não é uma string"
        
        linhas = resposta_content.strip().split('\n')
        
        if len(linhas) < 1:
             return False, "Resposta vazia após limpeza"
        
        primeira_linha = linhas[0].strip()
        if not any(campo in primeira_linha.lower() for campo in ['grupo', 'projeto', 'id', 'similaridade']):
            return False, f"Cabeçalho CSV não reconhecido. Linha encontrada: '{primeira_linha[:100]}...'"
        
        if len(linhas) > 1:
            segunda_linha = linhas[1].strip()
            # CORREÇÃO: Aceitar tanto vírgula quanto ponto e vírgula
            separadores = max(segunda_linha.count(','), segunda_linha.count(';'))
            if separadores < 3:  # Deve ter pelo menos 3 separadores (4 colunas)
                return False, "Formato de dados inválido na segunda linha"
        
        logging.info(f"✅ Resposta validada: {len(linhas)} linhas encontradas")
        return True, "Válida"
    
    except Exception as e:
        return False, f"Erro na validação: {e}"

# Função para limpar a resposta em CSV da LLM
def limpar_resposta_csv(resposta_content):
    """
    Limpa a resposta da API para extrair o conteúdo CSV.
    Remove marcadores de código, linhas em branco e texto explicativo.
    """
    try:
        # 1. Remove os marcadores de bloco de código e espaços em branco extras
        #    Isso lida com ```csv, ```, e qualquer texto antes/depois deles.
        if '```' in resposta_content:
            # Pega o conteúdo entre o primeiro ``` e o último ```
            partes = resposta_content.split('```')
            if len(partes) >= 2:
                # O conteúdo relevante geralmente está na segunda parte (índice 1)
                # Remove 'csv' se estiver no início da string
                csv_bruto = partes[1].lstrip('csv\n') 
            else:
                csv_bruto = resposta_content # Fallback
        else:
            csv_bruto = resposta_content # Se não houver ```, usa o conteúdo todo

        # 2. Remove linhas em branco no início e no fim
        linhas = csv_bruto.strip().split('\n')
        
        # 3. Encontra o início real do CSV (a linha do cabeçalho)
        inicio_csv = -1
        for i, linha in enumerate(linhas):
            linha_lower = linha.lower()
            # Procura por colunas essenciais para identificar o cabeçalho
            if 'grupo_id' in linha_lower and 'projeto_id' in linha_lower:
                inicio_csv = i
                break
        
        # Se não encontrar o cabeçalho, tenta uma abordagem mais genérica
        if inicio_csv == -1:
             for i, linha in enumerate(linhas):
                # Procura por uma linha que pareça dados CSV (com pelo menos 2 separadores)
                if linha.count(';') >= 2 or linha.count(',') >= 2:
                    inicio_csv = i
                    break
        
        # 4. Extrai apenas as linhas do CSV e junta tudo
        if inicio_csv != -1:
            linhas_csv_finais = [linha.strip() for linha in linhas[inicio_csv:] if linha.strip()]
            csv_limpo = '\n'.join(linhas_csv_finais)
            
            # 5. **NOVA CORREÇÃO**: Garante que a última linha termine com aspas se contiver uma
            if csv_limpo:
                ultima_linha = csv_limpo.split('\n')[-1]
                # Verifica se a última linha tem uma aspa de abertura mas não de fechamento
                if '"' in ultima_linha and not ultima_linha.endswith('"'):
                    csv_limpo += '"'
                    logging.warning("🔧 Corrigido CSV malformado: aspa de fechamento adicionada.")

            logging.info(f"🧹 CSV limpo com sucesso: {len(linhas_csv_finais)} linhas extraídas.")
            return csv_limpo
        else:
            logging.warning("⚠️ Cabeçalho CSV não encontrado na resposta. Retornando conteúdo bruto.")
            return resposta_content # Retorna original se não encontrar nada que pareça CSV

    except Exception as e:
        logging.error(f"❌ Erro crítico ao limpar CSV: {e}")
        return resposta_content # Retorna original em caso de erro

def monitorar_uso_api(resultados_processamento):
    try:
        total_requisicoes = len(resultados_processamento)
        total_projetos = sum(r['total_projetos'] for r in resultados_processamento)
        tokens_estimados = total_projetos * 300
        custo_estimado = (tokens_estimados / 1000) * 0.0014
        
        log_uso = {
            'timestamp': datetime.now().isoformat(),
            'total_requisicoes': total_requisicoes,
            'total_projetos': total_projetos,
            'tokens_estimados': tokens_estimados,
            'custo_estimado_usd': custo_estimado
        }
        
        os.makedirs('resultados_agrupamento/logs', exist_ok=True)
        arquivo_uso = 'resultados_agrupamento/logs/uso_api.json'
        with open(arquivo_uso, 'w', encoding='utf-8') as f:
            json.dump(log_uso, f, indent=2, ensure_ascii=False)
        
        logging.info(f"💰 Uso da API monitorado: {total_requisicoes} requests, ~${custo_estimado:.2f}")
        return log_uso
    
    except Exception as e:
        logging.error(f"❌ Erro ao monitorar uso da API: {e}")
        return None

# --- BLOCO DE EXECUÇÃO PRINCIPAL DO CHUNK 7 (TOTALMENTE REVISADO) ---

print("\n🔄 Executando Chunk 7: Validação, Limpeza e Salvamento")

# Inicializa a variável para evitar NameError
uso_api = None 
resultados_validos = []

if 'resultados_processamento' in locals() and resultados_processamento:
    print(f"🔍 Validando e limpando {len(resultados_processamento)} resultados...")
    
    resultados_invalidos = []
    
    for resultado in resultados_processamento:
        # 1. PRIMEIRO, limpa a resposta para extrair o CSV
        csv_limpo = limpar_resposta_csv(resultado['resposta'])
        
        # 2. DEPOIS, valida o conteúdo limpo
        valida, motivo = validar_resposta_api(csv_limpo)
        
        if valida:
            resultado['resposta_limpa'] = csv_limpo
            resultados_validos.append(resultado)
        else:
            logging.warning(f"⚠️ Resposta inválida para '{resultado['arquivo_saida']}': {motivo}")
            resultados_invalidos.append(resultado)
            # SALVA A RESPOSTA INVÁLIDA COMO .TXT PARA ANÁLISE POSTERIOR
            salvar_resultado_invalido_txt(resultado)
    
    # 3. SALVA os arquivos CSV que foram validados
    if resultados_validos:
        print(f"💾 Salvando {len(resultados_validos)} arquivos CSV de resultados...")
        arquivos_salvos = salvar_resultados_validados(resultados_validos)
        if arquivos_salvos:
            print(f"✅ {len(arquivos_salvos)} arquivos salvos com sucesso no diretório 'resultados_agrupamento/'.")
        
        # 4. MONITORA o uso da API apenas se houver sucesso
        uso_api = monitorar_uso_api(resultados_validos)
    
    print(f"\n✅ Chunk 7 executado:")
    print(f"   ✅ Respostas válidas e salvas: {len(resultados_validos)}")
    print(f"   ❌ Respostas inválidas: {len(resultados_invalidos)}")
    
    if uso_api:
        print(f"   💰 Custo estimado (apenas requisições válidas): ${uso_api['custo_estimado_usd']:.2f}")
else:
    print("⚠️ Chunk 7 ignorado: Nenhum resultado do Chunk 6 para validar.")

2025-08-18 13:15:42,544 - INFO - 🧹 CSV limpo com sucesso: 34 linhas extraídas.
2025-08-18 13:15:42,546 - INFO - ✅ Resposta validada: 34 linhas encontradas
2025-08-18 13:15:42,547 - INFO - 🧹 CSV limpo com sucesso: 38 linhas extraídas.
2025-08-18 13:15:42,548 - INFO - ✅ Resposta validada: 38 linhas encontradas
2025-08-18 13:15:42,549 - INFO - 🧹 CSV limpo com sucesso: 122 linhas extraídas.
2025-08-18 13:15:42,550 - INFO - ✅ Resposta validada: 122 linhas encontradas
2025-08-18 13:15:42,551 - INFO - 🧹 CSV limpo com sucesso: 55 linhas extraídas.
2025-08-18 13:15:42,552 - INFO - ✅ Resposta validada: 55 linhas encontradas
2025-08-18 13:15:42,552 - INFO - 🧹 CSV limpo com sucesso: 46 linhas extraídas.
2025-08-18 13:15:42,553 - INFO - ✅ Resposta validada: 46 linhas encontradas
2025-08-18 13:15:42,554 - INFO - 🧹 CSV limpo com sucesso: 127 linhas extraídas.
2025-08-18 13:15:42,555 - INFO - ✅ Resposta validada: 127 linhas encontradas
2025-08-18 13:15:42,555 - INFO - 🧹 CSV limpo com sucesso: 39 linha


🔄 Executando Chunk 7: Validação, Limpeza e Salvamento
🔍 Validando e limpando 351 resultados...


2025-08-18 13:15:42,745 - INFO - ✅ Resposta validada: 109 linhas encontradas
2025-08-18 13:15:42,746 - INFO - 🧹 CSV limpo com sucesso: 112 linhas extraídas.
2025-08-18 13:15:42,746 - INFO - ✅ Resposta validada: 112 linhas encontradas
2025-08-18 13:15:42,747 - INFO - 🧹 CSV limpo com sucesso: 125 linhas extraídas.
2025-08-18 13:15:42,748 - INFO - ✅ Resposta validada: 125 linhas encontradas
2025-08-18 13:15:42,748 - INFO - 🧹 CSV limpo com sucesso: 35 linhas extraídas.
2025-08-18 13:15:42,749 - INFO - ✅ Resposta validada: 35 linhas encontradas
2025-08-18 13:15:42,751 - INFO - 🧹 CSV limpo com sucesso: 13 linhas extraídas.
2025-08-18 13:15:42,751 - INFO - ✅ Resposta validada: 13 linhas encontradas
2025-08-18 13:15:42,752 - INFO - 🧹 CSV limpo com sucesso: 51 linhas extraídas.
2025-08-18 13:15:42,752 - INFO - ✅ Resposta validada: 51 linhas encontradas
2025-08-18 13:15:42,753 - INFO - 🧹 CSV limpo com sucesso: 47 linhas extraídas.
2025-08-18 13:15:42,754 - INFO - ✅ Resposta validada: 47 linhas e

💾 Salvando 349 arquivos CSV de resultados...


2025-08-18 13:15:43,269 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote24.csv
2025-08-18 13:15:43,270 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote25.csv
2025-08-18 13:15:43,271 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote26.csv
2025-08-18 13:15:43,272 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote27.csv
2025-08-18 13:15:43,273 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote28.csv
2025-08-18 13:15:43,274 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote29.csv
2025-08-18 13:15:43,275 - INFO - ✅ Arquivo CSV salvo: resultados_agrupamento/grupos_categoria_TIC_DE_-_Desenvolvimento

✅ 349 arquivos salvos com sucesso no diretório 'resultados_agrupamento/'.

✅ Chunk 7 executado:
   ✅ Respostas válidas e salvas: 349
   ❌ Respostas inválidas: 2
   💰 Custo estimado (apenas requisições válidas): $21.30


In [13]:
"""
CHUNK 8 (VERSÃO FINAL CORRIGIDA): Consolidação, Enriquecimento e Filtragem
Lê os CSVs salvos, combina-os corretamente com os dados multianuais e salva o resultado final com nome fixo.
"""
import pandas as pd
from io import StringIO
import os
import logging

def carregar_resultados_da_pasta():
    """
    Carrega todos os arquivos CSV de resultados parciais diretamente da pasta 'resultados_agrupamento'.
    """
    diretorio = 'resultados_agrupamento'
    dataframes_carregados = []
    
    print(f"🔎 Procurando por arquivos de resultado em '{diretorio}/'...")
    
    try:
        arquivos_no_diretorio = os.listdir(diretorio)
        arquivos_csv_resultados = [
            f for f in arquivos_no_diretorio 
            if f.startswith('grupos_categoria_') and f.endswith('.csv')
        ]
        
        if not arquivos_csv_resultados:
            logging.warning(f"⚠️ Nenhum arquivo CSV começando com 'grupos_categoria_' foi encontrado.")
            return []

        print(f"📄 Encontrados {len(arquivos_csv_resultados)} arquivos CSV de resultados para carregar.")
        
        for nome_arquivo in arquivos_csv_resultados:
            caminho_completo = os.path.join(diretorio, nome_arquivo)
            try:
                df = pd.read_csv(caminho_completo, sep=';', engine='python', quotechar='"', doublequote=True)
                dataframes_carregados.append(df)
                logging.info(f"  -> ✅ Carregado: '{nome_arquivo}' ({len(df)} linhas)")
            except Exception as e:
                # Loga o erro mas continua o processo com os outros arquivos
                logging.error(f"❌ Falha ao ler o arquivo '{nome_arquivo}', será ignorado. Erro: {e}")
                
        return dataframes_carregados

    except FileNotFoundError:
        print(f"❌ ERRO: O diretório '{diretorio}' não foi encontrado. Execute os Chunks 6 e 7 para gerar os arquivos.")
        return []

def consolidar_e_enriquecer_resultados(lista_dataframes_llm, df_processado):
    """
    Função principal que combina os resultados, agora incluindo a coluna 'sublote'.
    """
    try:
        logging.info("🔄 Iniciando consolidação e enriquecimento de dados...")
        
        # *** ATUALIZAÇÃO: Adiciona 'sublote' às colunas desejadas ***
        colunas_finais_desejadas = [
            'projeto_id', 'grupo_id_final', 'sublote', 'similaridade_score', 'justificativa_agrupamento', 
            'origem_agrupamento', 'ano_referencia', 'setor', 'natureza', 
            'tipo_pesquisa', 'empresa', 'projeto', 'do_id_at', 'do_resultado_analise'
        ]

        df_base = df_processado.copy()
        df_base['projeto_id'] = pd.to_numeric(df_base['projeto'].str.extract(r'ID ÚNICO:\s*(\d+)')[0], errors='coerce')
        df_base.dropna(subset=['projeto_id'], inplace=True)
        colunas_para_buscar = [c for c in colunas_finais_desejadas if c in df_base.columns]

        # 1. PROCESSAR RESULTADOS DA LLM
        df_llm_enriquecido = pd.DataFrame()
        if lista_dataframes_llm:
            df_llm_consolidado = pd.concat(lista_dataframes_llm, ignore_index=True)
            df_llm_consolidado.rename(columns={'grupo_id': 'grupo_id_temp'}, inplace=True)
            df_llm_consolidado['projeto_id'] = pd.to_numeric(df_llm_consolidado['projeto_id'], errors='coerce')
            
            df_llm_enriquecido = pd.merge(df_llm_consolidado, df_base[colunas_para_buscar], on='projeto_id', how='left')
            df_llm_enriquecido['origem_agrupamento'] = 'LLM'

        # 2. PROCESSAR DADOS MULTIANUAIS
        df_multianuais_formatado = pd.DataFrame()
        df_multianuais_ids = df_base[df_base['eh_multianual'] == True].copy()
        if not df_multianuais_ids.empty:
            df_multianuais_formatado = df_multianuais_ids[colunas_para_buscar + ['grupo_multianual']].copy()
            df_multianuais_formatado.rename(columns={'grupo_multianual': 'grupo_id_temp'}, inplace=True)
            df_multianuais_formatado['origem_agrupamento'] = 'MULTIANUAL_AUTO'
            df_multianuais_formatado['similaridade_score'] = 1.0 
            df_multianuais_formatado['justificativa_agrupamento'] = 'Agrupamento automático de projeto multianual'
            # *** ATUALIZAÇÃO: Adiciona um valor padrão para a coluna 'sublote' ***
            df_multianuais_formatado['sublote'] = 'N/A'

        # 3. CONSOLIDAR TUDO
        df_consolidado_final = pd.concat([df_llm_enriquecido, df_multianuais_formatado], ignore_index=True, sort=False)
        if df_consolidado_final.empty: return pd.DataFrame()

        # 4. RENUMERAÇÃO FINAL E LIMPEZA
        df_consolidado_final.drop_duplicates(subset=['projeto_id'], keep='first', inplace=True)
        df_consolidado_final['grupo_id_final'] = pd.factorize(df_consolidado_final['grupo_id_temp'])[0] + 1
        df_consolidado_final.loc[df_consolidado_final['grupo_id_temp'] == 0, 'grupo_id_final'] = 0
        
        colunas_presentes = [c for c in colunas_finais_desejadas if c in df_consolidado_final.columns]
        return df_consolidado_final[colunas_presentes]

    except Exception as e:
        logging.error(f"❌ Erro na consolidação completa: {e}")
        return None

# --- BLOCO DE EXECUÇÃO PRINCIPAL DO CHUNK 8 ---
print("\n🔄 Executando Chunk 8: Consolidação a partir de Arquivos")

df_consolidado = None # Reset da variável
df_final_agrupados = pd.DataFrame() # Garante que a variável exista

try:
    # 1. Carrega todos os CSVs de resultados da pasta
    lista_dfs_llm = carregar_resultados_da_pasta()
    
    # 2. Consolida os dados carregados com os multianuais
    df_consolidado = consolidar_e_enriquecer_resultados(lista_dfs_llm, df_processado)
    
    if df_consolidado is not None and not df_consolidado.empty:
        # 3. Atribui o resultado final à variável. A filtragem já foi feita na lógica.
        df_final_agrupados = df_consolidado
        
        # 4. Salva o arquivo final com nome fixo
        # *** NOME DO ARQUIVO FIXO, SEM TIMESTAMP ***
        arquivo_final = 'resultados_agrupamento/GRUPOS_FINAL_FILTRADO.csv'
        df_final_agrupados.to_csv(arquivo_final, index=False, encoding='utf-8', sep=';')
        
        print(f"✅ Chunk 8 executado com sucesso!")
        print(f"📄 Arquivo final salvo com {len(df_final_agrupados)} projetos em: {arquivo_final}")
        print(f"   - Grupos da LLM: {len(df_final_agrupados[df_final_agrupados['origem_agrupamento'] == 'LLM'])}")
        print(f"   - Grupos Multianuais: {len(df_final_agrupados[df_final_agrupados['origem_agrupamento'] == 'MULTIANUAL_AUTO'])}")
    else:
        print("❌ Falha na consolidação ou nenhum grupo foi formado. Nenhum dado foi processado para o arquivo final.")

except Exception as e:
    print(f"❌ Ocorreu um erro inesperado no Chunk 8: {e}")

2025-08-18 13:16:49,338 - INFO -   -> ✅ Carregado: 'grupos_categoria_Transversal_DE_-_Desenvolvimento_Expe_Processo_sublote8.csv' (54 linhas)
2025-08-18 13:16:49,343 - INFO -   -> ✅ Carregado: 'grupos_categoria_Química_e_Farmácia_DE_-_Desenvolvimento_Expe_Produto_sublote34.csv' (116 linhas)
2025-08-18 13:16:49,349 - INFO -   -> ✅ Carregado: 'grupos_categoria_Metalurgia_e_Mineração_DE_-_Desenvolvimento_Expe_Processo_sublote2.csv' (70 linhas)
2025-08-18 13:16:49,354 - INFO -   -> ✅ Carregado: 'grupos_categoria_Agroindústria_e_Alimentos_DE_-_Desenvolvimento_Expe_Produto_sublote3.csv' (106 linhas)
2025-08-18 13:16:49,358 - INFO -   -> ✅ Carregado: 'grupos_categoria_Metalurgia_e_Mineração_DE_-_Desenvolvimento_Expe_Produto_sublote1.csv' (59 linhas)
2025-08-18 13:16:49,362 - INFO -   -> ✅ Carregado: 'grupos_categoria_Transversal_PA_-_Pesquisa_Aplicada_Processo_sublote1.csv' (39 linhas)
2025-08-18 13:16:49,367 - INFO -   -> ✅ Carregado: 'grupos_categoria_Agroindústria_e_Alimentos_DE_-_Desenvol

2025-08-18 13:16:49,392 - INFO -   -> ✅ Carregado: 'grupos_categoria_Eletroeletrônica_PA_-_Pesquisa_Aplicada_Produto_sublote6.csv' (58 linhas)
2025-08-18 13:16:49,396 - INFO -   -> ✅ Carregado: 'grupos_categoria_Química_e_Farmácia_PA_-_Pesquisa_Aplicada_Produto_sublote6.csv' (102 linhas)
2025-08-18 13:16:49,399 - INFO -   -> ✅ Carregado: 'grupos_categoria_Agroindústria_e_Alimentos_DE_-_Desenvolvimento_Expe_Produto_sublote1.csv' (65 linhas)
2025-08-18 13:16:49,403 - INFO -   -> ✅ Carregado: 'grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote6.csv' (47 linhas)
2025-08-18 13:16:49,406 - INFO -   -> ✅ Carregado: 'grupos_categoria_Química_e_Farmácia_DE_-_Desenvolvimento_Expe_Produto_sublote16.csv' (46 linhas)
2025-08-18 13:16:49,410 - INFO -   -> ✅ Carregado: 'grupos_categoria_Transversal_DE_-_Desenvolvimento_Expe_Produto_sublote16.csv' (133 linhas)
2025-08-18 13:16:49,413 - INFO -   -> ✅ Carregado: 'grupos_categoria_Transversal_PA_-_Pesquisa_Aplicada_Produto_sublote4.csv' (112


🔄 Executando Chunk 8: Consolidação a partir de Arquivos
🔎 Procurando por arquivos de resultado em 'resultados_agrupamento/'...
📄 Encontrados 369 arquivos CSV de resultados para carregar.


2025-08-18 13:16:49,532 - INFO -   -> ✅ Carregado: 'grupos_categoria_Química_e_Farmácia_PB_-_Pesquisa_Básica_Processo_completa.csv' (40 linhas)
2025-08-18 13:16:49,534 - INFO -   -> ✅ Carregado: 'grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Produto_sublote18.csv' (130 linhas)
2025-08-18 13:16:49,537 - INFO -   -> ✅ Carregado: 'grupos_categoria_Agroindústria_e_Alimentos_PB_-_Pesquisa_Básica_Serviço_completa.csv' (4 linhas)
2025-08-18 13:16:49,539 - INFO -   -> ✅ Carregado: 'grupos_categoria_Metalurgia_e_Mineração_DE_-_Desenvolvimento_Expe_Produto_sublote9.csv' (95 linhas)
2025-08-18 13:16:49,547 - INFO -   -> ✅ Carregado: 'grupos_categoria_Metalurgia_e_Mineração_DE_-_Desenvolvimento_Expe_Processo_sublote12.csv' (138 linhas)
2025-08-18 13:16:49,550 - INFO -   -> ✅ Carregado: 'grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Processo_sublote4.csv' (102 linhas)
2025-08-18 13:16:49,553 - INFO -   -> ✅ Carregado: 'grupos_categoria_TIC_DE_-_Desenvolvimento_Expe_Serviço_sublote9.csv' (115 linh

✅ Chunk 8 executado com sucesso!
📄 Arquivo final salvo com 34823 projetos em: resultados_agrupamento/GRUPOS_FINAL_FILTRADO.csv
   - Grupos da LLM: 27201
   - Grupos Multianuais: 7622


In [21]:
"""
CHUNK 9 (MODIFICADO): Validação e Qualidade dos Grupos
Valida a qualidade dos agrupamentos e gera métricas de avaliação.
"""

# As funções auxiliares são adaptadas para usar 'grupo_id_final'

def analisar_qualidade_grupos(df_agrupado):
    try:
        if df_agrupado.empty: return None
        distribuicao_tamanhos = df_agrupado.groupby('grupo_id_final').size()
        return {'distribuicao_grupos': {'total_grupos': len(distribuicao_tamanhos), 'tamanho_medio': distribuicao_tamanhos.mean(), 'maior_grupo': distribuicao_tamanhos.max()}}
    except Exception as e:
        logging.error(f"❌ Erro na análise de qualidade: {e}")
        return None

def gerar_amostras_grupos(df_agrupado, num_amostras=5):
    try:
        if df_agrupado.empty: return []
        amostras = []
        grupos_validos = df_agrupado[df_agrupado['grupo_id_final'] > 0]
        ids_grupos_amostra = grupos_validos['grupo_id_final'].drop_duplicates().sample(min(num_amostras, grupos_validos['grupo_id_final'].nunique()))
        for grupo_id in ids_grupos_amostra:
            projetos_grupo = grupos_validos[grupos_validos['grupo_id_final'] == grupo_id]
            amostras.append({'grupo_id': int(grupo_id), 'tamanho_grupo': len(projetos_grupo), 'projetos': projetos_grupo[['projeto_id', 'ano_referencia', 'setor']].to_dict('records')})
        return amostras
    except Exception as e:
        logging.error(f"❌ Erro ao gerar amostras: {e}")
        return []

def calcular_metricas_agrupamento(df_agrupado, df_total):
    try:
        total_projetos = len(df_total)
        projetos_agrupados = len(df_agrupado)
        total_grupos = df_agrupado['grupo_id_final'].nunique()
        metricas = {
            'cobertura': projetos_agrupados / total_projetos if total_projetos > 0 else 0,
            'densidade_media': projetos_agrupados / total_grupos if total_grupos > 0 else 0
        }
        return metricas
    except Exception as e:
        logging.error(f"❌ Erro ao calcular métricas: {e}")
        return None

def gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp):
    # Função mantida para salvar os relatórios
    try:
        relatorio_completo = {'metadata': {'timestamp': timestamp}, 'qualidade': relatorio_qualidade, 'metricas': metricas, 'amostras': amostras}
        arquivo_relatorio = f'resultados_agrupamento/relatorio_validacao_{timestamp}.json'
        with open(arquivo_relatorio, 'w', encoding='utf-8') as f: json.dump(relatorio_completo, f, indent=2, ensure_ascii=False, default=str)
        return arquivo_relatorio
    except Exception as e:
        logging.error(f"❌ Erro ao gerar relatório de validação: {e}")
        return None

# --- BLOCO DE EXECUÇÃO PRINCIPAL DO CHUNK 9 (MODIFICADO) ---
print("\n🔄 Executando Chunk 9: Validação e Qualidade")

# A validação agora é feita sobre o df_final_agrupados do Chunk 8
if 'df_final_agrupados' in locals() and df_final_agrupados is not None and not df_final_agrupados.empty:
    print(f"🔍 Analisando a qualidade de {len(df_final_agrupados)} projetos agrupados...")
    
    # Passamos o df_consolidado original para calcular a cobertura total corretamente
    metricas = calcular_metricas_agrupamento(df_final_agrupados, df_consolidado)
    relatorio_qualidade = analisar_qualidade_grupos(df_final_agrupados)
    amostras = gerar_amostras_grupos(df_final_agrupados)
    
    relatorio_path = gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp_final)
    
    print(f"✅ Chunk 9 executado:")
    if metricas:
        print(f"   📈 Cobertura (proporção de projetos que formaram grupos): {metricas.get('cobertura', 0):.1%}")
    if relatorio_qualidade:
        print(f"   🏷️ Total de grupos formados: {relatorio_qualidade['distribuicao_grupos']['total_grupos']}")
    print(f"   📋 Relatório de validação salvo em: {relatorio_path}")
else:
    print("⚠️ Chunk 9 ignorado: Nenhum projeto agrupado encontrado no Chunk 8 para analisar.")


🔄 Executando Chunk 9: Validação e Qualidade
🔍 Analisando a qualidade de 19531 projetos agrupados...
✅ Chunk 9 executado:
   📈 Cobertura (proporção de projetos que formaram grupos): 100.0%
   🏷️ Total de grupos formados: 7472
   📋 Relatório de validação salvo em: resultados_agrupamento/relatorio_validacao_20250815_114303.json


In [ ]:
"""
CHUNK 10: Execução Completa e Resumo Final
Função principal que executa todo o pipeline adaptado e gera o relatório final.
"""

def executar_pipeline_completo(df_original, modo_teste=True, limite_teste=3, categoria_teste=None):
    """
    Executa todo o pipeline de agrupamento de projetos com a lógica atualizada de 2025.
    
    Args:
        df_original (pd.DataFrame): O DataFrame inicial carregado do CSV.
        modo_teste (bool): Se True, executa apenas um número limitado de itens.
        limite_teste (int): O número de itens a processar em modo teste.
        categoria_teste (str, optional): Filtra para processar apenas uma categoria.
    """
    try:
        timestamp_inicio = datetime.now()
        timestamp_str = timestamp_inicio.strftime('%Y%m%d_%H%M%S')
        
        logging.info(f"🚀 INICIANDO PIPELINE COMPLETO (Timestamp: {timestamp_str})")
        
        resultados_pipeline = {
            'inicio': timestamp_inicio.isoformat(),
            'modo_teste': modo_teste,
            'categoria_teste': categoria_teste,
            'etapas': {}
        }

        # ===== ETAPA 1: PREPARAÇÃO DOS DADOS (Lógica do Chunk 2) =====
        print("\n" + "="*60 + "\nETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS\n" + "="*60)
        global CATEGORIA_TESTE_API
        CATEGORIA_TESTE_API = categoria_teste
        df_processado, df_para_llm, combinacoes_validas, grupos_multianuais = preparar_dados_com_multianual(df_original)
        resultados_pipeline['etapas']['1_preparacao'] = {'registros_para_llm': len(df_para_llm)}
        print(f"✅ Etapa 1 concluída.")

        # ===== ETAPA 2: PROCESSAMENTO COM IA (Chunks 4, 5, 6) =====
        print("\n" + "="*60 + "\nETAPA 2: PROCESSAMENTO DOS AGRUPAMENTOS COM IA\n" + "="*60)
        system_message_template = criar_system_message()
        plano_processamento = preparar_plano_processamento_categoria(df_para_llm, combinacoes_validas)
        resultados_processamento = executar_processamento_sincronizado(
            plano_processamento, system_message_template, modo_teste=modo_teste, limite_teste=limite_teste
        )
        resultados_pipeline['etapas']['2_processamento_ia'] = {'respostas_recebidas': len(resultados_processamento)}
        print(f"✅ Etapa 2 concluída.")

        # ===== ETAPA 3: VALIDAÇÃO E SALVAMENTO (Lógica do Chunk 7) =====
        print("\n" + "="*60 + "\nETAPA 3: VALIDAÇÃO E SALVAMENTO DOS RESULTADOS PARCIAIS\n" + "="*60)
        resultados_validos = []
        if resultados_processamento:
            for resultado in resultados_processamento:
                csv_limpo = limpar_resposta_csv(resultado['resposta'])
                valida, _ = validar_resposta_api(csv_limpo)
                if valida:
                    resultado['resposta_limpa'] = csv_limpo
                    resultados_validos.append(resultado)
            salvar_resultados_validados(resultados_validos)
        resultados_pipeline['etapas']['3_validacao'] = {'respostas_validas': len(resultados_validos)}
        print(f"✅ Etapa 3 concluída.")

        # ===== ETAPA 4: CONSOLIDAÇÃO E FILTRAGEM (Lógica do Chunk 8) =====
        print("\n" + "="*60 + "\nETAPA 4: CONSOLIDAÇÃO, ENRIQUECIMENTO E FILTRAGEM\n" + "="*60)
        resultados_com_df = carregar_resultados_para_merge(resultados_validos)
        resultados_mergeados = processar_merge_completo(resultados_com_df)
        df_consolidado = consolidar_e_enriquecer_resultados(resultados_mergeados, df_processado, grupos_multianuais)
        
        df_final_agrupados = pd.DataFrame() # Garante que a variável exista
        if df_consolidado is not None and not df_consolidado.empty:
            df_final_agrupados = df_consolidado[df_consolidado['grupo_id_final'] > 0].copy()
            arquivo_final = f'resultados_agrupamento/GRUPOS_FINAL_FILTRADO.csv'
            df_final_agrupados.to_csv(arquivo_final, index=False, encoding='utf-8', sep=';')
            resultados_pipeline['etapas']['4_consolidacao'] = {'arquivo_final': arquivo_final, 'projetos_agrupados': len(df_final_agrupados)}
            print(f"✅ Etapa 4 concluída.")
        else:
             resultados_pipeline['etapas']['4_consolidacao'] = {'erro': 'DataFrame consolidado vazio'}

        # ===== ETAPA 5: ANÁLISE DE QUALIDADE (Lógica do Chunk 9) =====
        print("\n" + "="*60 + "\nETAPA 5: ANÁLISE DE QUALIDADE E RELATÓRIO FINAL\n" + "="*60)
        if not df_final_agrupados.empty:
            metricas = calcular_metricas_agrupamento(df_final_agrupados, df_consolidado) # Passa ambos para calcular cobertura
            relatorio_qualidade = analisar_qualidade_grupos(df_final_agrupados)
            amostras = gerar_amostras_grupos(df_final_agrupados)
            gerar_relatorio_validacao(relatorio_qualidade, metricas, amostras, timestamp_str)
            resultados_pipeline['etapas']['5_analise_qualidade'] = {'metricas': metricas}
            print(f"✅ Etapa 5 concluída.")
        else:
            print("⚠️ Nenhuma análise de qualidade pois não houve grupos formados.")
            resultados_pipeline['etapas']['5_analise_qualidade'] = {'metricas': None}

        # ===== FINALIZAÇÃO =====
        timestamp_fim = datetime.now()
        resultados_pipeline['duracao_total_segundos'] = (timestamp_fim - timestamp_inicio).total_seconds()
        return resultados_pipeline

    except Exception as e:
        logging.error(f"❌ ERRO CRÍTICO NO PIPELINE: {e}")
        resultados_pipeline['erro_geral'] = str(e)
        return resultados_pipeline

def gerar_resumo_execucao(resultados_pipeline):
    print("\n" + "="*80 + "\n🎉 RESUMO FINAL DA EXECUÇÃO DO PIPELINE\n" + "="*80)
    
    duracao_s = resultados_pipeline.get('duracao_total_segundos', 0)
    print(f"⏰ Duração total: {duracao_s:.2f} segundos")
    
    if '4_consolidacao' in resultados_pipeline['etapas'] and resultados_pipeline['etapas']['4_consolidacao'].get('arquivo_final'):
        stats_consol = resultados_pipeline['etapas']['4_consolidacao']
        print(f"\n🔗 RESULTADO PRINCIPAL:")
        print(f"  - {stats_consol.get('projetos_agrupados', 0)} projetos foram agrupados.")
        print(f"  - Arquivo principal salvo em: {stats_consol.get('arquivo_final', 'N/A')}")

    if '5_analise_qualidade' in resultados_pipeline['etapas'] and resultados_pipeline['etapas']['5_analise_qualidade'].get('metricas'):
        metricas = resultados_pipeline['etapas']['5_analise_qualidade']['metricas']
        print(f"\n📈 MÉTRICAS FINAIS:")
        print(f"  - Cobertura de Agrupamento: {metricas.get('cobertura', 0):.1%}")
        print(f"  - Densidade Média dos Grupos: {metricas.get('densidade_media', 0):.1f} projetos/grupo")
    
    if resultados_pipeline.get('erro_geral'):
        print(f"\n❌ ATENÇÃO: O pipeline foi interrompido por um erro grave: {resultados_pipeline['erro_geral']}")
    else:
        print("\n🎉 PIPELINE EXECUTADO COM SUCESSO!")

# --- BLOCO DE EXECUÇÃO PRINCIPAL DO CHUNK 10 ---

print("\n🚀 INICIANDO A EXECUÇÃO COMPLETA DO PIPELINE...")

# Para executar em modo teste (3 primeiros sublotes da categoria de teste):
resultados_finais = executar_pipeline_completo(
    df, 
    modo_teste=True, 
    limite_teste=3, 
    categoria_teste="Metalurgia e Mineração"
)

# Para executar o pipeline completo em todos os dados (pode demorar e custar dinheiro):
# resultados_finais = executar_pipeline_completo(df, modo_teste=False)

# Gerar o resumo da execução
gerar_resumo_execucao(resultados_finais)

2025-08-12 15:35:56,299 - INFO - 🚀 INICIANDO PIPELINE COMPLETO (Timestamp: 20250812_153556)
2025-08-12 15:35:56,429 - INFO - 📊 Dados iniciais limpos: 74466 registros



🚀 INICIANDO A EXECUÇÃO COMPLETA DO PIPELINE...

ETAPA 1: CARREGAMENTO E PREPARAÇÃO DOS DADOS


2025-08-12 15:35:57,405 - INFO - 📊 Dados extraídos:
2025-08-12 15:35:57,413 - INFO -    📋 CNPJs únicos: 5717
2025-08-12 15:35:57,422 - INFO -    🏢 Razões sociais únicas: 5669
2025-08-12 15:35:57,475 - INFO -    📄 Projetos multianuais: 54529
2025-08-12 15:35:58,452 - INFO - 🔗 Projetos multianuais identificados:
2025-08-12 15:35:58,454 - INFO -    📊 Grupos multianuais: 7434
2025-08-12 15:35:58,455 - INFO -    📋 Total de registros multianuais: 17979
2025-08-12 15:35:58,508 - INFO -    📄 Exemplo 1: Desenvolvimento de novas soluções voltadas ao aten... (3 anos: [np.float64(2020.0), np.float64(2021.0), np.float64(2023.0)])
2025-08-12 15:35:58,509 - INFO -    📄 Exemplo 2: Desenvolvimento de soluções de atendimento integra... (5 anos: [np.float64(2019.0), np.float64(2020.0), np.float64(2021.0), np.float64(2022.0), np.float64(2023.0)])
2025-08-12 15:35:58,510 - INFO -    📄 Exemplo 3: Desenvolvimento de soluções para atendimento de cl... (3 anos: [np.float64(2020.0), np.float64(2021.0), np.float

✅ Etapa 1 concluída.

ETAPA 2: PROCESSAMENTO DOS AGRUPAMENTOS COM IA


2025-08-12 15:36:03,708 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-08-12 15:36:03,789 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-08-12 15:36:03,970 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-08-12 15:37:25,786 - INFO - ✅ Resposta assíncrona recebida para Química e Farmácia
2025-08-12 15:37:31,726 - INFO - ✅ Resposta assíncrona recebida para Química e Farmácia
2025-08-12 15:37:37,190 - INFO - ✅ Resposta assíncrona recebida para Química e Farmácia
2025-08-12 15:37:37,191 - INFO - ✅ Lote assíncrono concluído: 3/3 sucessos
2025-08-12 15:37:37,192 - INFO - ✅ Lote 1 concluído: 3 sucessos
2025-08-12 15:37:37,194 - INFO - 
🎉 Processamento assíncrono concluído!
2025-08-12 15:37:37,195 - INFO - ✅ Sucessos: 3/139
2025-08-12 15:37:37,196 - INFO - 🧹 CSV limpo: 70 linhas
2025-08-12 15:37:37,197 - INFO - ✅ Resposta validada: 70 linhas encontradas
2025-0

✅ Etapa 2 concluída.

ETAPA 3: VALIDAÇÃO E SALVAMENTO DOS RESULTADOS PARCIAIS
✅ Etapa 3 concluída.

ETAPA 4: CONSOLIDAÇÃO, ENRIQUECIMENTO E FILTRAGEM
✅ Etapa 4 concluída.

ETAPA 5: ANÁLISE DE QUALIDADE E RELATÓRIO FINAL
✅ Etapa 5 concluída.

🎉 RESUMO FINAL DA EXECUÇÃO DO PIPELINE
⏰ Duração total: 101.53 segundos

🔗 RESULTADO PRINCIPAL:
  - 81 projetos foram agrupados.
  - Arquivo principal salvo em: resultados_agrupamento/GRUPOS_FINAL_FILTRADO_20250812_153556.csv

📈 MÉTRICAS FINAIS:
  - Cobertura de Agrupamento: 0.4%
  - Densidade Média dos Grupos: 4.0 projetos/grupo

🎉 PIPELINE EXECUTADO COM SUCESSO!
